In [ ]:
pip install Pillow pandas

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def create_certificate(template_path, output_folder, name, email):
    """
    Create a certificate for a single student
    """
    img = Image.open(template_path)
    draw = ImageDraw.Draw(img)
    
    try:
        font = ImageFont.truetype("arial.ttf", 60)
    except:
        font = ImageFont.load_default()
    
    W, H = img.size
    text_bbox = draw.textbbox((0, 0), name, font=font)
    text_width = text_bbox[2] - text_bbox[0]
    text_x = (W - text_width) / 2
    text_y = 650
    
    draw.text((text_x, text_y), name, font=font, fill='black')
    
    os.makedirs(output_folder, exist_ok=True)
    
    output_path = os.path.join(output_folder, f'certificate_{name}.png')
    img.save(output_path)
    return output_path

def send_email(recipient, subject, body, attachment_path):
    """
    Send email with certificate attachment
    """
    sender_email = ""
    sender_password = ""

    try:
        # Set up the SMTP server
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.starttls()
        server.login(sender_email, sender_password)

        # Create message
        msg = MIMEMultipart()
        msg["From"] = sender_email
        msg["To"] = recipient
        msg["Subject"] = subject
        msg.attach(MIMEText(body, "plain"))

        # Attach the certificate
        with open(attachment_path, "rb") as attachment:
            part = MIMEBase("application", "octet-stream")
            part.set_payload(attachment.read())
            encoders.encode_base64(part)
            filename = os.path.basename(attachment_path)
            part.add_header("Content-Disposition", f'attachment; filename="{filename}"')
            msg.attach(part)

        # Send email
        server.sendmail(sender_email, recipient, msg.as_string())
        logging.info(f"✅ Email sent successfully to {recipient}")
        
    except Exception as e:
        logging.error(f"❌ Error sending email to {recipient}: {e}")
        raise
    
    finally:
        server.quit()

def process_certificates(template_path, csv_path, output_folder):
    """
    Generate and email certificates for all students in the CSV file
    """
    # Read the CSV file
    df = pd.read_csv(csv_path)
    
    for _, row in df.iterrows():
        try:
            # Generate certificate
            certificate_path = create_certificate(
                template_path,
                output_folder,
                row['name'],
                row['email']
            )
            logging.info(f"Created certificate for {row['name']}")

            # Prepare email content
            subject = "Certificate of Participation - CUK Workshop"
            body = f"""Dear {row['name']},

Thank you for participating in the One-Day Workshop on "Bureau of Energy Efficiency (BEE)" held at the Central University of Karnataka.

Please find attached your certificate of participation.

Best regards,
CUK Workshop Team"""

            # Send email
            send_email(row['email'], subject, body, certificate_path)
            
        except Exception as e:
            logging.error(f"Failed to process certificate for {row['name']}: {e}")
            continue

if __name__ == "__main__":
    # Configuration
    template_path = "CUK_temp.png"  # Your certificate template
    csv_path = "students_data.csv"  # Your CSV file with student data
    output_folder = "generated_certificates"
    
    try:
        process_certificates(template_path, csv_path, output_folder)
        logging.info("Certificate generation and email process completed")
    except Exception as e:
        logging.error(f"Process failed: {e}")